In [1]:
import cv2
import keyboard as key
import os
import numpy as np
import pandas as pd
from PIL import Image
path = 'cannyout/'

In [2]:
def calculateMetricAll(image):
    # average of All pixels
    im = Image.open(image)
    im_grey = im.convert('LA') # convert to grayscale
    #im_grey = im
    width, height = im.size

    total = 0
    for i in range(0, width-1):
        for j in range(0, height-1):
            total += im_grey.getpixel((i,j))[0]

    mean = total / (width * height)
    #print(mean)
    return mean

In [3]:
def calculateMetricNonBlack(image):
    # average of All pixels
    im = Image.open(image)
    im_grey = im.convert('LA') # convert to grayscale
    #im_grey = im
    width, height = im.size
    total = 0
    totalpix = 0
    for i in range(0, width):
        for j in range(0, height):
            pixval = im_grey.getpixel((i,j))[0]
            if pixval != 0:
                total += pixval
                totalpix += 1
                #print(pixval)

    mean = total / totalpix
    #print(mean)
    return mean

In [ ]:
files = os.listdir(path)
lisall = []
lisnonblack = []
i=1
for file in files:
    avgall = calculateMetricAll(path+file)
    lisall.append(avgall)
    avgnonblack = calculateMetricNonBlack(path+file)
    lisnonblack.append(avgnonblack)
print(lisall)
print(lisnonblack)

In [ ]:
labels = ['filename','meanAll','meanNonBlack']
print(type(labels))
filenames = os.listdir(path)
print(type(filenames))

In [ ]:
df = pd.DataFrame()
df[labels[0]] = filenames
df[labels[1]] = lisall
df[labels[2]] = lisnonblack
df.head()

In [ ]:
df.to_csv('matric.csv')

In [6]:
# addonly to non-black pixels 
# if v>255 v=255
# if v<0 v=0
# im[row:rowend, col:col:colend] += value
def brightness(file,avg):
    im = Image.open(file)
    im_grey = im.convert('LA')
    im_new = Image.new('L',im.size)
    width, height = im.size
    for i in range(0, width):
        for j in range(0, height):
            pixval = im_grey.getpixel((i,j))[0]
            if pixval != 0 and pixval<255:
                pixval += avg
                im_new.putpixel((i,j),pixval)
                #totalpix += 1
                #print(pixval)
            elif pixval+avg >= 255:
                im_new.putpixel((i,j),255)
    return im_new

# im = brightness(path+'1.jpg',int(df['meanNonBlack'][0].item()))
# im.show()

In [ ]:
# length = len(files)
# # i=0
# # import keyboard
# # while True:
# #     if keyboard.is_pressed('n'):
        
# #         i += 1
# #     elif keyboard.is_pressed('e'):
# #         break
# import keyboard
# def on_press_reaction(event):
#     global i
#     if event.name == 'n':
#         im = brightness(path+files[i],int(df['meanNonBlack'][i].item()))
#         im.show()
#         i += 1

# keyboard.on_press(on_press_reaction)

# while True:
#     pass

In [11]:
df = pd.read_csv('matric.csv')
df.head()
filenames = df['filename']
lisnonblack = df['meanNonBlack']
avg = sum(lisnonblack)/len(lisnonblack)
print(avg)

99.76405956839109


In [17]:
length = len(filenames)
n = 0
for i in range(0,length):
    im = brightness(path+filenames[i],int(avg))
    im.save('testout/'+filenames[i])